# ¿Qué es el One Hot Encoding? ¿Por qué y cuando debemos usarlo?

One hot encoding es un proceso en el cual variables categoiricas son convertidas en una forma en la que se la podemos entregar a un algoritmo de *Machine Learning* para hacer un mejor trabajo al momento de predecir
is a process by which categorical variables are converted into a form that could be provided to ML algorithms to do a better job in prediction.

Veamos, estas jugnado con un modelo de ML y te encientras con esto del *“One hot encoding”* en todos lados, haces lo que siempre haces en estos casos y revisas la [documenacion](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html?ref=hackernoon.com) de sklearn para *one hot encoder* y dice: “Encode categorical integer features using a one-hot aka one-of-K scheme.” ,i.e, no dice nada, esto es lo malo de las docuimentaciones, pueden ser un poco "circulares" a ratos, así que veamos un ejemplo de que es lo que es esto:

### Supongamos el siguiente dataset:

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'company_name':['VW', 'Acura', 'Honda', 'Honda'], 'categorical_value':[1,1,2,3], 'price':[20000, 10011, 50000, 10000]})
df

,company_name,categorical_value,price
0,VW,1,20000
1,Acura,1,10011
2,Honda,2,50000
3,Honda,3,10000


La variable `categorical_value` representa el valor numerico de la entrada en el dataset, i.e., si hubiera una cuarta compañia, esta tendría un valor de 4. De forma que el numero de valores únicos aumenta, los valroes categoricos también lo hacen. 

Podemos usar [*labelEncoder*](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html?ref=hackernoon.com) de sklearn para generar esta columna en base a una columna de valores categoricos (aunque este iría de 0 a n-1)

Ahora, centrandonos en lo que nos interesa, si seguimos las intruscciones que nos dan en la documentacion que citamos al principio, obtendríamos el siguiente dataset:

In [3]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder(handle_unknown='ignore')
enc_df = pd.DataFrame(enc.fit_transform(df[['company_name']]).toarray())
oh_df = df.join(enc_df)
oh_df.columns = list(oh_df.columns[:3])+['is_Acura', 'is_Honda', 'is_VW'] #esto no es necesario, es solo con el fin de ilustrar el ejemplo
oh_df

,company_name,categorical_value,price,is_Acura,is_Honda,is_VW
0,VW,1,20000,0.0,0.0,1.0
1,Acura,1,10011,1.0,0.0,0.0
2,Honda,2,50000,0.0,1.0,0.0
3,Honda,3,10000,0.0,1.0,0.0


0 indica que no existe mientras que 1 indica que si lo hace.

Antes de seguir, ¿se les ocurre porque usar la columnas *categorical_value* no es suficiente?

El problema con esto es que implica un orden, asume que una categoria con un mayor valor numerico es una "mejor categoría"

A qué me refiero con esto, esta forma de organizacion asume qué VW > Acura > Honda based basado en los valores categoricos, asumamos que el modelo en algun momento desea tomar un promedio, entonces nos daría 1+3 = 4/2 =2. Esto implica que el promedio de VW y Honda es Acura. Creo que todos podemos estar de acuerdo que hay algo mal con eso. Este modelo terminaría por perder capacidad dado este error.

Este es el motivo por el cual usamos *one hot encoder* para “binarizar” los datos de las categorias para incluirlas en nuestro modelo

Ahora, veamos un ejemplo un poco más robusto. Usando una base de datos de cancer de mama

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/breast-cancer.csv"
dataset = pd.read_csv(url, header=None)
dataset.head()

,0,1,2,3,4,5,6,7,8,9
0,'40-49','premeno','15-19','0-2','yes','3','right','left_up','no','recurrence-events'
1,'50-59','ge40','15-19','0-2','no','1','right','central','no','no-recurrence-events'
2,'50-59','ge40','35-39','0-2','no','2','left','left_low','no','recurrence-events'
3,'40-49','premeno','35-39','0-2','yes','3','right','left_low','yes','no-recurrence-events'
4,'40-49','premeno','30-34','3-5','yes','2','left','right_up','no','recurrence-events'


Vamos a rescatar los datos como array para que sean un poco más fáciles de trabajar. También usaremos `LabelEncoder`, acá pueden ver su [documentación](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html), pero el tl;dr, hace lo mismo pero crea una sola columna para "esto pasa o no pasa"

In [5]:
data = dataset.values
# Los separamos entre input y output
X = data[:, :-1].astype(str)
y = data[:, -1].astype(str)
# creamos el one hot encoder 
onehot_encoder = OneHotEncoder(sparse=False)
X = onehot_encoder.fit_transform(X)
# label con
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)
# summarize the transformed data
print('Input', X.shape)
print(X[:5, :], '\n\n',y[:5])

Input (286, 43)
[[0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.
  0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  1. 0. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0.]] 

 [1 0 1 0 1]


Ahora, vamos a hacer una regresión logística rápida para nuestro modelo. 

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import accuracy_score

# separamos los datos en train y test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# one-hot encode 
onehot_encoder = OneHotEncoder()
onehot_encoder.fit(X_train)
X_train = onehot_encoder.transform(X_train)
X_test = onehot_encoder.transform(X_test)
# ordinal encode variable objetivo
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train = label_encoder.transform(y_train)
y_test = label_encoder.transform(y_test)
# definimos el modelo
model = LogisticRegression()
# hacemos el fit
model.fit(X_train, y_train)
# predecimos
yhat = model.predict(X_test)
# evaluamos
accuracy = accuracy_score(y_test, yhat)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 69.47


No es el mejor modelo del mundo, pero hay muchos otros que podedríamos usar, pero lo que quiero vean es que pudimos transdormar un datates basicamente inutul en uno que podemos meter en un modelo, ahí les va a ustedes en su imaginación que es lo que pueden hacer. Dicho esto, ahora viene eso:

### Actividades
#### Actividad 1

Para este ejercico usaremos los datos de kaggle de (House Prices: Advanced Regression Techniques)[https://www.kaggle.com/c/house-prices-advanced-regression-techniques/overview]. Es una competencia, así que les recomiendo mirarla y participar si es que se animan! Les entregaré los datos amononados, de forma de centrarnos en lo que nos interesa en este momento. Filtraremos todos los datos que no son categoricos (el criterio será más de 10 valores únicos), y borraremos las columnas con datos nulos.

In [7]:
import pandas as pd
train_data = pd.read_csv('./data/train.csv')
test_data = pd.read_csv('./data/test.csv')

# vamos a dropear donde no tengamos target
train_data.dropna(axis=0, subset=['SalePrice'], inplace=True)

target = train_data.SalePrice

# Como el objetivo no es manejar valores nan, y es un ejemplo
# vamos a dropear las columnas con valores faltantes
# si quieren aprender más sobre el tema recomiendo
# https://www.kaggle.com/dansbecker/handling-missing-values
cols_with_missing = [col for col in train_data.columns 
                                 if train_data[col].isnull().any()]                                  
candidate_train_predictors = train_data.drop(['Id', 'SalePrice'] + cols_with_missing, axis=1)

# "cardinality" es el numero de valores únicos
# Lo usamos para seleccionar columnas categoricos
# Esto es conveniente pero quizás algo arbitrario
low_cardinality_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].nunique() < 10 and
                                candidate_train_predictors[cname].dtype == "object"]
numeric_cols = [cname for cname in candidate_train_predictors.columns if 
                                candidate_train_predictors[cname].dtype in ['int64', 'float64']]
my_cols = low_cardinality_cols + numeric_cols
predictors = candidate_train_predictors[my_cols]
predictors.head()

,MSZoning,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,Condition1,Condition2,BldgType,...,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,RL,Pave,Reg,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,548,0,61,0,0,0,0,0,2,2008
1,RL,Pave,Reg,Lvl,AllPub,FR2,Gtl,Feedr,Norm,1Fam,...,460,298,0,0,0,0,0,0,5,2007
2,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Norm,Norm,1Fam,...,608,0,42,0,0,0,0,0,9,2008
3,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,Norm,Norm,1Fam,...,642,0,35,272,0,0,0,0,2,2006
4,RL,Pave,IR1,Lvl,AllPub,FR2,Gtl,Norm,Norm,1Fam,...,836,192,84,0,0,0,0,0,12,2008


Ya con los datos limpios, separen los datos (usen `random_state=42` y `test_size=0.33`) y utilizen el *encoder* para transformar los datos: 

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
X = predictors
y = target
# separamos los datos en train y test 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1)
# one-hot encode 
onehot_encoder = OneHotEncoder()
onehot_encoder.fit(X)
X_train = onehot_encoder.transform(X_train)
X_test = onehot_encoder.transform(X_test)

Ahora, usen un modelo a elección (ojalá no una regresión logística) para modelar las precios, y vean que tal les va

In [9]:
X_test

<482x6969 sparse matrix of type '<class 'numpy.float64'>'
	with 27474 stored elements in Compressed Sparse Row format>

In [10]:
y_test

258     231500
267     179500
288     122000
649      84500
1233    142000
         ...  
1115    318000
825     385000
937     253000
1387    136000
764     270000
Name: SalePrice, Length: 482, dtype: int64

In [11]:
from sklearn.neighbors import NearestCentroid
from sklearn.metrics import r2_score

model =  NearestCentroid()
# hacemos el fit
model.fit(X_train, y_train)
# predecimos
yhat = model.predict(X_test)
# evaluamos
accuracy = r2_score(y_test, yhat)
print('R2: %.2f' % (accuracy)) #no es el mejor puntaje, pero lo importante es haber podido usar el o-h-e

R2: 0.65


Genial, ahora les toca un no tan grande desafío

#### Actividad 2: Ganenme

Ahora que ya saben como funciona esto, les dejo el dataframe para que intenten ganarme. No debería ser muy dificil dado el modelo que ocupé, pero las unicas reglas es que deben usar los mismos parámetros para `train_test_split` y no repetir modelos que ya se hayan usado en este notebook. Suerte!

In [ ]:
import pandas as pd
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/breast-cancer.csv"
df = pd.read_csv(url, header=None)
df.head()